In [ ]:
from google.colab import drive
import os

# Mount Drive
drive.mount('/content/drive')

# Create checkpoint directory for MFCC experiments
checkpoint_dir = '/content/drive/MyDrive/voiceflow_mfcc_checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)
print(f"✅ MFCC Checkpoints will be saved to: {checkpoint_dir}")

## Step 2: Install Dependencies

In [ ]:
%%capture
# Install required packages (includes librosa for MFCC extraction)
!pip install -q datasets[audio] torch torchaudio transformers accelerate librosa soundfile

# Note: librosa provides MFCC extraction and delta computation

In [ ]:
# Verify GPU
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")

## Step 3: Clone Model Code from GitHub

In [ ]:
import os

# Check if already cloned
repo_root = '/content/VoiceFlow-Intelligence-Platform'
if os.path.exists(repo_root):
    print("✅ Repository already exists, pulling latest changes...")
    !cd /content/VoiceFlow-Intelligence-Platform && git pull origin main
else:
    print("Cloning repository...")
    %cd /content
    !git clone https://github.com/FCHEHIDI/VoiceFlow-Intelligence-Platform.git
    
# Verify the structure and find the correct path
print("\n🔍 Detecting repository structure...")

# Colab sometimes creates deeply nested structures. Check all possibilities:
possible_paths = [
    '/content/VoiceFlow-Intelligence-Platform/voiceflow-ml/VoiceFlow-Intelligence-Platform/voiceflow-ml',  # Double-nested
    '/content/VoiceFlow-Intelligence-Platform/VoiceFlow-Intelligence-Platform/voiceflow-ml',  # Nested
    '/content/VoiceFlow-Intelligence-Platform/voiceflow-ml',  # Flat
]

voiceflow_ml_path = None
for path in possible_paths:
    models_path = os.path.join(path, 'models')
    if os.path.exists(path) and os.path.exists(models_path):
        voiceflow_ml_path = path
        print(f"✅ Found voiceflow-ml with models/ at: {path}")
        break

if not voiceflow_ml_path:
    # Debugging: show actual structure at each level
    print(f"\n❌ Could not find voiceflow-ml with models/ directory")
    print(f"\n📂 /content/VoiceFlow-Intelligence-Platform/:")
    !ls -la /content/VoiceFlow-Intelligence-Platform/
    
    # Check voiceflow-ml directory
    if os.path.exists('/content/VoiceFlow-Intelligence-Platform/voiceflow-ml'):
        print(f"\n📂 /content/VoiceFlow-Intelligence-Platform/voiceflow-ml/:")
        !ls -la /content/VoiceFlow-Intelligence-Platform/voiceflow-ml/
        
        # Check for nested repo inside voiceflow-ml
        nested_repo = '/content/VoiceFlow-Intelligence-Platform/voiceflow-ml/VoiceFlow-Intelligence-Platform'
        if os.path.exists(nested_repo):
            print(f"\n📂 Found nested repo at: {nested_repo}")
            print(f"Contents:")
            !ls -la {nested_repo}
    
    raise FileNotFoundError("Could not find voiceflow-ml/models/ - see directory listings above")

# Verify critical files exist
models_path = os.path.join(voiceflow_ml_path, 'models')
diarization_path = os.path.join(models_path, 'diarization')
model_file = os.path.join(diarization_path, 'model.py')

print(f"✅ models/ directory: {models_path}")
print(f"✅ models/diarization/ directory: {diarization_path}")
print(f"✅ models/diarization/model.py: {'exists' if os.path.exists(model_file) else 'MISSING'}")

if not os.path.exists(model_file):
    print(f"\n❌ ERROR: model.py not found!")
    print(f"\nContents of models/diarization/:")
    !ls -la {diarization_path}
    raise FileNotFoundError(f"model.py not found at {model_file}")

# Store the path for next cells
%store voiceflow_ml_path
print(f"\n✅ Repository verified successfully!")
print(f"📁 Using: {voiceflow_ml_path}")

## Step 4: Load Streaming Dataset (Zero Local Storage!)

In [ ]:
from datasets import load_dataset, Audio

# Dataset selection
DATASET_NAME = "librispeech_asr"
print(f"Loading {DATASET_NAME} in STREAMING mode...")
print("⚠️ No download required - audio streams on-demand!\n")

# Load train split (streaming)
train_dataset = load_dataset(
    DATASET_NAME,
    "clean",  # LibriSpeech subset
    split="train.360",  # 360 hours
    streaming=True,
    trust_remote_code=True
)

# Load validation split
val_dataset = load_dataset(
    DATASET_NAME,
    "clean",
    split="validation",
    streaming=True,
    trust_remote_code=True
)

# Cast audio column to ensure proper decoding
train_dataset = train_dataset.cast_column("audio", Audio(sampling_rate=16000, decode=True))
val_dataset = val_dataset.cast_column("audio", Audio(sampling_rate=16000, decode=True))

print("✅ Datasets ready (streaming mode)")
print(f"Local storage used: 0 GB ✅\n")

# Inspect first sample
try:
    sample = next(iter(train_dataset))
    print("Sample keys:", sample.keys())
    print(f"Audio shape: {sample['audio']['array'].shape}")
    print(f"Sample rate: {sample['audio']['sampling_rate']} Hz")
except Exception as e:
    print(f"⚠️ Sample inspection failed: {e}")
    print("This is OK - audio will decode when training starts")

## Step 5: Load Enhanced Model with MFCC Support

In [ ]:
import sys
import os

# Retrieve the voiceflow_ml_path from Step 3
try:
    %store -r voiceflow_ml_path
    print(f"✅ Using voiceflow-ml path from Step 3: {voiceflow_ml_path}")
except:
    # Fallback: try to find it again
    print("⚠️ Path not stored, searching...")
    possible_paths = [
        '/content/VoiceFlow-Intelligence-Platform/voiceflow-ml',
        '/content/VoiceFlow-Intelligence-Platform/VoiceFlow-Intelligence-Platform/voiceflow-ml',
    ]
    voiceflow_ml_path = None
    for path in possible_paths:
        if os.path.exists(os.path.join(path, 'models')):
            voiceflow_ml_path = path
            break
    
    if not voiceflow_ml_path:
        raise FileNotFoundError("Could not find voiceflow-ml directory. Please re-run Step 3.")

# Add to Python path
if voiceflow_ml_path not in sys.path:
    sys.path.insert(0, voiceflow_ml_path)

print(f"✅ Python path configured: {voiceflow_ml_path}")

# Verify models directory exists
models_path = os.path.join(voiceflow_ml_path, 'models')
if not os.path.exists(models_path):
    print(f"\n❌ ERROR: models/ directory not found!")
    print(f"Expected location: {models_path}")
    print(f"\nContents of {voiceflow_ml_path}:")
    !ls -la {voiceflow_ml_path}
    raise FileNotFoundError(f"models/ directory not found at {models_path}")

print(f"✅ models/ directory found: {models_path}")

# Now import the model
try:
    from models.diarization.model import FastDiarizationModel
    print("✅ Model imported successfully")
except ImportError as e:
    print(f"\n❌ Import failed: {e}")
    print(f"\nDebugging info:")
    print(f"  Current directory: {os.getcwd()}")
    print(f"  Python sys.path[0]: {sys.path[0]}")
    print(f"  models/ exists: {os.path.exists(models_path)}")
    print(f"  models/__init__.py exists: {os.path.exists(os.path.join(models_path, '__init__.py'))}")
    print(f"  models/diarization/ exists: {os.path.exists(os.path.join(models_path, 'diarization'))}")
    print(f"  models/diarization/model.py exists: {os.path.exists(os.path.join(models_path, 'diarization/model.py'))}")
    print(f"\nContents of models/:")
    !ls -la {models_path}
    raise

# Model configuration with MFCC input channels
CONFIG = {
    'num_speakers': 2,
    'hidden_size': 256,
    'encoder_type': 'lightweight-cnn',
    'dropout': 0.3,
    'in_channels': 80,  # 🔑 NEW: 40 MFCCs + 40 deltas
}

# Initialize model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = FastDiarizationModel(**CONFIG).to(device)

print(f"\n{'='*60}")
print(f"Model: FastDiarizationModel (Enhanced CNN with MFCC)")
print(f"Device: {device}")
print(f"Input: 80 channels (40 MFCCs + 40 deltas)")
print(f"Parameters: {model.count_parameters() / 1e6:.2f}M")
print(f"Trainable: {model.count_trainable_parameters() / 1e6:.2f}M")
print(f"{'='*60}\n")

## Step 6: Create MFCC Streaming DataLoader

In [ ]:
import torch
import torchaudio
from torch.utils.data import DataLoader, IterableDataset
import numpy as np
import librosa

class StreamingAudioDataset(IterableDataset):
    """
    Wrapper for streaming HuggingFace dataset with MFCC feature extraction.
    
    Extracts 40 MFCC coefficients + 40 delta coefficients = 80 features.
    This replaces raw waveform input with acoustic features for better accuracy.
    """
    
    def __init__(self, hf_dataset, target_sr=16000, duration=3.0, max_samples=None):
        self.dataset = hf_dataset
        self.target_sr = target_sr
        self.target_length = int(target_sr * duration)
        self.max_samples = max_samples
        
        # MFCC parameters (industry standard)
        self.n_mfcc = 40  # 40 coefficients
        self.n_fft = 512  # FFT window
        self.hop_length = 160  # 10ms hop @ 16kHz
        self.n_mels = 80  # Mel filterbank size
        
        # Pre-scan dataset to create persistent label mapping
        print("🔍 Pre-scanning dataset to create stable label mapping...")
        self.speaker_to_label = {}
        self._create_label_mapping()
        print(f"✅ Mapped {len(self.speaker_to_label)} speakers to 2 labels")
        print(f"   Label 0: {sum(1 for l in self.speaker_to_label.values() if l == 0)} speakers")
        print(f"   Label 1: {sum(1 for l in self.speaker_to_label.values() if l == 1)} speakers")
    
    def _create_label_mapping(self):
        """Pre-scan dataset once to build stable speaker-to-label mapping."""
        next_label = 0
        count = 0
        
        for sample in self.dataset:
            if self.max_samples and count >= self.max_samples:
                break
            
            speaker_id = str(sample.get('speaker_id', 0))
            if speaker_id not in self.speaker_to_label:
                self.speaker_to_label[speaker_id] = next_label
                next_label = (next_label + 1) % CONFIG['num_speakers']
            
            count += 1
    
    def _extract_mfcc_features(self, audio_np):
        """
        Extract MFCC features from audio waveform.
        
        Args:
            audio_np: numpy array of audio samples (1D)
        
        Returns:
            torch.Tensor of shape (80, n_frames):
                - 40 MFCC coefficients
                - 40 delta coefficients
        """
        # Extract 40 MFCCs
        mfcc = librosa.feature.mfcc(
            y=audio_np,
            sr=self.target_sr,
            n_mfcc=self.n_mfcc,
            n_fft=self.n_fft,
            hop_length=self.hop_length,
            n_mels=self.n_mels
        )  # Shape: (40, n_frames)
        
        # Compute delta features (temporal dynamics)
        mfcc_delta = librosa.feature.delta(mfcc)  # Shape: (40, n_frames)
        
        # Stack to create 80-channel input
        features = np.vstack([mfcc, mfcc_delta])  # Shape: (80, n_frames)
        
        return torch.FloatTensor(features)
    
    def __iter__(self):
        count = 0
        for sample in self.dataset:
            # Stop if max_samples reached
            if self.max_samples and count >= self.max_samples:
                break
            
            try:
                # Extract audio
                audio = torch.FloatTensor(sample['audio']['array'])
                sr = sample['audio']['sampling_rate']
                
                # Resample to 16kHz if needed
                if sr != self.target_sr:
                    resampler = torchaudio.transforms.Resample(sr, self.target_sr)
                    audio = resampler(audio)
                
                # Pad/crop to fixed length
                if audio.shape[0] > self.target_length:
                    # Random crop
                    start = np.random.randint(0, audio.shape[0] - self.target_length)
                    audio = audio[start:start + self.target_length]
                elif audio.shape[0] < self.target_length:
                    # Zero-pad
                    audio = torch.nn.functional.pad(audio, (0, self.target_length - audio.shape[0]))
                
                # 🔑 NEW: Extract MFCC features instead of raw waveform
                audio_np = audio.numpy()
                features = self._extract_mfcc_features(audio_np)
                # features shape: (80, ~300 frames) for 3-second audio
                
                # Use persistent label mapping
                speaker_id = str(sample.get('speaker_id', 0))
                label = self.speaker_to_label.get(speaker_id, 0)
                
                yield features, label
                count += 1
                
            except Exception as e:
                # Skip corrupted samples
                print(f"Skipping sample due to error: {e}")
                continue

# Create streaming datasets with MFCC extraction
print("🎵 Creating MFCC streaming datasets...")
train_streaming = StreamingAudioDataset(train_dataset, max_samples=10000)
val_streaming = StreamingAudioDataset(val_dataset, max_samples=1000)

# Create dataloaders
BATCH_SIZE = 32

def collate_fn(batch):
    """Collate function for 2D MFCC features."""
    features, labels = zip(*batch)
    
    # Pad features to same length (handle variable frame counts)
    max_frames = max(f.shape[1] for f in features)
    padded_features = []
    
    for f in features:
        if f.shape[1] < max_frames:
            # Pad frames dimension
            pad_amount = max_frames - f.shape[1]
            f = torch.nn.functional.pad(f, (0, pad_amount))
        padded_features.append(f)
    
    return torch.stack(padded_features), torch.LongTensor(labels)

train_loader = DataLoader(
    train_streaming,
    batch_size=BATCH_SIZE,
    collate_fn=collate_fn,
    num_workers=0
)

val_loader = DataLoader(
    val_streaming,
    batch_size=BATCH_SIZE,
    collate_fn=collate_fn,
    num_workers=0
)

print("✅ MFCC DataLoaders ready (streaming mode)")
print(f"   Features: 40 MFCCs + 40 deltas = 80 channels")
print(f"   Distribution: Balanced round-robin assignment")

## Step 7: Training Configuration

In [ ]:
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR

# Training hyperparameters
TRAINING_CONFIG = {
    'num_epochs': 30,
    'learning_rate': 1e-3,
    'weight_decay': 1e-4,
    'warmup_epochs': 3,
    'save_every_n_epochs': 5,
    'validate_every_n_epochs': 1,
}

# Optimizer
optimizer = optim.AdamW(
    model.parameters(),
    lr=TRAINING_CONFIG['learning_rate'],
    weight_decay=TRAINING_CONFIG['weight_decay']
)

# Learning rate scheduler
scheduler = CosineAnnealingLR(
    optimizer,
    T_max=TRAINING_CONFIG['num_epochs'],
    eta_min=1e-6
)

# Loss function
criterion = torch.nn.CrossEntropyLoss()

print("✅ Training configuration ready")
print(f"\nHyperparameters:")
for key, value in TRAINING_CONFIG.items():
    print(f"  {key}: {value}")

## Step 8: Checkpoint Resume Logic

In [ ]:
import glob

def load_latest_checkpoint(checkpoint_dir, model, optimizer, scheduler=None):
    """
    Load the latest checkpoint if available.
    Returns: start_epoch, best_val_acc
    """
    checkpoints = glob.glob(f"{checkpoint_dir}/checkpoint_epoch*.pth")
    
    if not checkpoints:
        print("No checkpoint found. Starting from scratch.")
        return 0, 0.0
    
    # Find latest checkpoint
    latest_checkpoint = max(checkpoints, key=os.path.getctime)
    print(f"Loading checkpoint: {latest_checkpoint}")
    
    checkpoint = torch.load(latest_checkpoint, map_location=device)
    
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    
    if scheduler and 'scheduler_state_dict' in checkpoint:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    
    start_epoch = checkpoint['epoch'] + 1
    best_val_acc = checkpoint.get('best_val_acc', 0.0)
    
    print(f"✅ Resumed from epoch {checkpoint['epoch']}")
    print(f"   Best validation accuracy: {best_val_acc:.2f}%\n")
    
    return start_epoch, best_val_acc

# Load checkpoint if exists
start_epoch, best_val_acc = load_latest_checkpoint(
    checkpoint_dir, model, optimizer, scheduler
)

## Step 9: Training Loop

In [ ]:
from tqdm.notebook import tqdm
import time

def train_epoch(model, train_loader, criterion, optimizer, device, epoch):
    """
    Train for one epoch.
    """
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0
    num_batches = 0
    
    pbar = tqdm(train_loader, desc=f"[MFCC] Epoch {epoch+1}/{TRAINING_CONFIG['num_epochs']}")
    
    for batch_idx, (features, labels) in enumerate(pbar):
        # Move to GPU
        features = features.to(device)
        labels = labels.to(device)
        
        # Forward pass
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Metrics
        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        num_batches += 1
        
        # Update progress bar
        pbar.set_postfix({
            'loss': total_loss / num_batches,
            'acc': 100.0 * correct / total
        })
    
    avg_loss = total_loss / num_batches
    accuracy = 100.0 * correct / total
    
    return avg_loss, accuracy


def validate(model, val_loader, criterion, device):
    """
    Validate model.
    """
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    num_batches = 0
    
    with torch.no_grad():
        for features, labels in tqdm(val_loader, desc="[MFCC] Validation"):
            features = features.to(device)
            labels = labels.to(device)
            
            outputs = model(features)
            loss = criterion(outputs, labels)
            
            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            num_batches += 1
    
    avg_loss = total_loss / num_batches
    accuracy = 100.0 * correct / total
    
    return avg_loss, accuracy


# Training loop
print("\n" + "="*60)
print("Starting MFCC Training")
print("Expected: Val Acc 75-80% (vs 61.5% raw waveform baseline)")
print("="*60 + "\n")

training_start = time.time()

for epoch in range(start_epoch, TRAINING_CONFIG['num_epochs']):
    epoch_start = time.time()
    
    # Train
    train_loss, train_acc = train_epoch(
        model, train_loader, criterion, optimizer, device, epoch
    )
    
    # Validate
    val_loss, val_acc = None, None
    if (epoch + 1) % TRAINING_CONFIG['validate_every_n_epochs'] == 0:
        val_loss, val_acc = validate(model, val_loader, criterion, device)
        
        # Update best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_path = f"{checkpoint_dir}/best_model.pth"
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'val_acc': val_acc,
            }, best_model_path)
            print(f"✅ New best model saved! Val Acc: {val_acc:.2f}%")
    
    # Update learning rate
    scheduler.step()
    
    # Epoch summary
    epoch_time = time.time() - epoch_start
    print(f"\n[MFCC] Epoch {epoch+1}/{TRAINING_CONFIG['num_epochs']} Summary:")
    print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
    if val_acc is not None:
        print(f"  Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc:.2f}%")
    print(f"  Time: {epoch_time:.1f}s | LR: {scheduler.get_last_lr()[0]:.2e}\n")
    
    # Save checkpoint
    if (epoch + 1) % TRAINING_CONFIG['save_every_n_epochs'] == 0:
        checkpoint_path = f"{checkpoint_dir}/checkpoint_epoch{epoch+1}.pth"
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'train_loss': train_loss,
            'val_acc': val_acc if val_acc else 0.0,
            'best_val_acc': best_val_acc,
        }, checkpoint_path)
        print(f"💾 Checkpoint saved: {checkpoint_path}\n")

training_time = time.time() - training_start
print("\n" + "="*60)
print("MFCC Training Complete!")
print("="*60)
print(f"Total time: {training_time/3600:.2f} hours")
print(f"Best validation accuracy: {best_val_acc:.2f}%")
print(f"Improvement over raw waveform: +{best_val_acc - 61.5:.2f}%")
print(f"Best model saved at: {checkpoint_dir}/best_model.pth")

## Step 10: Export Trained Model to ONNX

In [ ]:
# Load best model
best_model_path = f"{checkpoint_dir}/best_model.pth"
checkpoint = torch.load(best_model_path, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

print(f"Loaded best MFCC model (Val Acc: {checkpoint['val_acc']:.2f}%)\n")

# Export to ONNX
dummy_input = torch.randn(1, 80, 300).to(device)  # 80 channels, ~300 frames
onnx_path = f"{checkpoint_dir}/fast_cnn_mfcc_trained.onnx"

torch.onnx.export(
    model,
    dummy_input,
    onnx_path,
    input_names=['mfcc_features'],
    output_names=['speaker_logits'],
    dynamic_axes={
        'mfcc_features': {0: 'batch_size', 2: 'n_frames'},
        'speaker_logits': {0: 'batch_size'}
    },
    opset_version=14
)

print(f"✅ ONNX model exported: {onnx_path}")
print(f"Model size: {os.path.getsize(onnx_path) / (1024**2):.2f} MB")

# Download ONNX model
from google.colab import files
files.download(onnx_path)

## 🎉 MFCC Training Complete!

**Results Summary**:
- Input: 80-channel MFCC features (40 coefficients + 40 deltas)
- Expected Val Acc: 75-80% (vs 61.5% raw waveform)
- Training time: ~2 hours (same as raw waveform)

**Next Steps**:
1. Compare MFCC vs raw waveform performance
2. Analyze convergence speed and stability
3. Update TRAINING_RESULTS_SUMMARY.md with MFCC results
4. If Val Acc ≥75%, proceed to production deployment
5. If Val Acc <75%, consider X-vectors or AMI Corpus

**Key Improvements**:
- ✅ Industry-standard acoustic features
- ✅ Better generalization (Train-Val gap should be <10%)
- ✅ Faster convergence (peak around epoch 15-20)
- ✅ More stable training (lower variance)